In [1]:
#import statements
import json
import ipywidgets as widgets
import plotly
import pandas as pd
# !pip install anytree
from anytree import PostOrderIter
from anytree.importer import DictImporter


In [2]:
#read file
with open('flare.json') as f:
    js_data = json.loads(f.read())

In [3]:
#imports dictonary in a tree form
importer = DictImporter()
root = importer.import_(js_data)


size = []
name = []
parent = []
level = []

def format(node):
  for i in node.children:
    #check if node as attribute value
    if hasattr(i, 'value') == False:
      format(i)
    v = i.value
    #check if node parent as attribute value
    if hasattr(i.parent, 'value'):
      i.parent.value += v
    #if node parent doesn't have a value set to same val as child
    elif hasattr(i.parent, 'value')== False:
      i.parent.value = v

    level.append(len(i.ancestors))
    name.append(i.name)
    parent.append(i.parent.name)
    size.append(i.value)
    
format(root)

#append attributes for root
level.append(0)
name.append(root.name)
parent.append("")
size.append(root.value)

#create df
df = pd.DataFrame()
df['parent'] = parent
df['name'] = name
df['value']= size
df['level'] = level

#slider funtion
def update(sliderVal):
  fig = plotly.graph_objs.Figure()
  fig.add_trace(plotly.graph_objs.Treemap(
      labels = df[df['level']<sliderVal]['name'],
      values = df[df['level']<sliderVal]['value'],
      parents = df[df['level']<sliderVal]['parent']  
  ))
  fig.update_traces(root_color="#f1f1f1")
  fig.update_layout(width = 900, height = 900)
  fig.show()

#create slider widget
widgets.interact(update, sliderVal = (0, 5))

interactive(children=(IntSlider(value=2, description='sliderVal', max=5), Output()), _dom_classes=('widget-int…

<function __main__.update(sliderVal)>

In [10]:
import ipywidgets as widgets
import pandas as pd
from datetime import datetime

start_date = datetime(2023, 3, 1)
end_date = datetime(2023, 3, 31)

dates = pd.date_range(start_date, end_date, freq='D')

options = [(date.strftime('%Y-%m-%d'), date) for date in dates]
index = (0, len(options)-1)

selection_range_slider = widgets.SelectionRangeSlider(
    options=options,
    index=index,
    description='Dates',
    orientation='horizontal',
    layout={'width': '700px'}
)

selection_range_slider


SelectionRangeSlider(description='Dates', index=(0, 30), layout=Layout(width='700px'), options=(('2023-03-01',…

In [4]:
import plotly.graph_objects as go
import datetime

# Create some dummy data to plot
x = [datetime.datetime(2023, 2, i) for i in range(1, 28)]
y = [i**2 for i in range(1, 28)]

# Create a figure object
fig = go.Figure()

# Add a scatter plot
fig.add_trace(go.Scatter(x=x, y=y, mode='lines'))

# Add a slider widget
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1d", step="day", stepmode="backward"),
                dict(count=7, label="1w", step="day", stepmode="backward"),
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True,
            range=[datetime.datetime(2023, 2, 1),
                   datetime.datetime(2023, 3, 1)]
        ),
        type="date"
    )
)

# Show the plot
fig.show()


In [5]:
import plotly.express as px

# Create a sample dataset
df = px.data.gapminder().query("year==2007")

# Create a scatter plot
fig = px.scatter(df, x="gdpPercap", y="lifeExp", color="continent", size="pop",
                 hover_name="country", log_x=True, size_max=60)

# Convert the scatter plot to a treemap
fig.update_traces(
    textposition='top center',
    textfont=dict(size=12),
    selector=dict(type='scatter')
).update_layout(
    margin=dict(t=10, b=10, r=10, l=10),
    hovermode='closest',
    updatemenus=[dict(type='buttons',
                      showactive=False,
                      buttons=[dict(label='Zoom In',
                                    method='zoom',
                                    args=[None, dict(frame=dict(width=200, height=200))]),
                               dict(label='Zoom Out',
                                    method='zoom',
                                    args=[None, dict(frame=dict(width=600, height=400))])])],
    xaxis=dict(showgrid=False, zeroline=False, visible=False),
    yaxis=dict(showgrid=False, zeroline=False, visible=False),
    autosize=False,
    width=600,
    height=400
).update_layout(
    treemap=dict(
        branchvalues='total',
        labels=df['continent'],
        parents=['world']*len(df['continent']),
        values=df['pop'],
        outsidetextfont={"size": 20, "color": "darkblue"},
        insidetextfont={"size": 20, "color": "white"},
    )
)

# Show the treemap
fig.show()


ValueError: 
    Invalid value of type 'builtins.str' received for the 'method' property of layout.updatemenu.button
        Received value: 'zoom'

    The 'method' property is an enumeration that may be specified as:
      - One of the following enumeration values:
            ['restyle', 'relayout', 'animate', 'update', 'skip']

In [11]:
from datetime import datetime
import pandas as pd
import ipywidgets as widgets

start_date = datetime(2021, 7, 1)
end_date = datetime(2021, 7, 5)

dates = pd.date_range(start_date, end_date, freq='D')

options = {date.strftime(' %d %b %Y '): date for date in dates}

pick_start = widgets.DatePicker(
    description='',
    disabled=False,
    value=start_date
)
pick_end = start_day = widgets.DatePicker(
    description='',
    disabled=False,
    value=end_date
)

selection_range_slider = widgets.SelectionRangeSlider(
    options=options,
    index=(0, len(options) - 1),
    description='Period',
    orientation='horizontal',
    layout=widgets.Layout(width='100%', padding='35px')
)


def update_pick(*args):
    pick_start.value = selection_range_slider.value[0]
    pick_end.value = selection_range_slider.value[1]


def update_slider(*args):
    selection_range_slider.value = (pick_start.value, pick_end.value)


selection_range_slider.observe(update_pick, 'value')
pick_start.observe(update_slider, 'value')
pick_end.observe(update_slider, 'value')

center_layout = widgets.Layout(display='flex',
                                       align_items='center',
                                       width='100%')

day = widgets.HBox(
    children=[pick_start, selection_range_slider, pick_end], layout=center_layout)

display(day)


/Users/sunlingfeng/litao/study-project/.venv/lib/python3.8/site-packages/traitlets/traitlets.py:714: FutureWarning:

Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.

/Users/sunlingfeng/litao/study-project/.venv/lib/python3.8/site-packages/ipywidgets/widgets/widget_selection.py:317: FutureWarning:

Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.



TraitError: Invalid selection: value not found

TraitError: Invalid selection: value not found

In [12]:
import ipywidgets as widgets
import datetime

# Define the start and end dates
start_date = datetime.datetime(2023, 2, 1)
end_date = datetime.datetime(2023, 3, 1)

# Create a date range slider widget
date_range_slider = widgets.DateRangeSlider(
    value=[start_date, end_date],
    min=start_date,
    max=end_date,
    step=datetime.timedelta(days=1),
    description='Date Range:',
    layout=widgets.Layout(width='80%')
)

# Create a function to handle the slider value change


def handle_date_range_change(change):
    start_date = change.new[0]
    end_date = change.new[1]
    print(f'Start Date: {start_date}, End Date: {end_date}')


# Attach the function to the slider's value change event
date_range_slider.observe(handle_date_range_change, names='value')

# Show the slider widget
date_range_slider


AttributeError: module 'ipywidgets' has no attribute 'DateRangeSlider'

In [18]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display

class DateRangePicker(object):
    def __init__(self,start,end,freq='D',fmt='%Y/%m/%d'):
        """
        Parameters
        ----------
        start : string or datetime-like
            Left bound of the period
        end : string or datetime-like
            Left bound of the period
        freq : string or pandas.DateOffset, default='D'
            Frequency strings can have multiples, e.g. '5H' 
        fmt : string, defauly = '%Y-%m-%d'
            Format to use to display the selected period

        """
        self.date_range=pd.date_range(start=start,end=end,freq=freq)
        options = [(item.strftime(fmt),item) for item in self.date_range]
        self.slider_start = widgets.SelectionSlider(
            description='start',
            options=options,
            continuous_update=False
        )
        self.slider_end = widgets.SelectionSlider(
            description='end',
            options=options,
            continuous_update=False,
            value=options[-1][1]
        )

        self.slider_start.on_trait_change(self.slider_start_changed, 'value')
        self.slider_end.on_trait_change(self.slider_end_changed, 'value')

        self.widget = widgets.Box(children=[self.slider_start,self.slider_end])

    def slider_start_changed(self,key,value):
        self.slider_end.value=max(self.slider_start.value,self.slider_end.value)
        self._observe(start=self.slider_start.value,end=self.slider_end.value)

    def slider_end_changed(self,key,value):
        self.slider_start.value=min(self.slider_start.value,self.slider_end.value)
        self._observe(start=self.slider_start.value,end=self.slider_end.value)

    def display(self):
        display(self.slider_start,self.slider_end)

    def _observe(self,**kwargs):
        if hasattr(self,'observe'):
            self.observe(**kwargs)

def fct(start,end):
    print(start,end)

In [20]:
w = DateRangePicker(start='2016-08-02', end="2016-09-02",
                    freq='D', fmt='%m-%d')
w.observe = fct
w.display()


/var/folders/my/x1pz96mj4_l31xr74gb254fh0000gn/T/ipykernel_18004/63436940.py:34: DeprecationWarning:

on_trait_change is deprecated in traitlets 4.1: use observe instead

/var/folders/my/x1pz96mj4_l31xr74gb254fh0000gn/T/ipykernel_18004/63436940.py:35: DeprecationWarning:

on_trait_change is deprecated in traitlets 4.1: use observe instead



SelectionSlider(continuous_update=False, description='start', options=(('08-02', Timestamp('2016-08-02 00:00:0…

SelectionSlider(continuous_update=False, description='end', index=31, options=(('08-02', Timestamp('2016-08-02…

2016-09-02 00:00:00 2016-09-02 00:00:00
